In [1]:
from constants import *
from data_loading import *
from data_cleaning import *
#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)


# load training set as DataFrame
train = load_df(TRAIN_NEG, TRAIN_POS, TWEET, LABEL, LABEL_NEG, LABEL_POS)
# set patterns to remove, replace, and replace with
to_remove = "<user>"
to_replace = "[^a-zA-Z#]"
replace_value = " "
# clean training set
train = clean(train, TWEET, CLEAN_TWEET, to_remove, to_replace, replace_value)

In [2]:
train.head()

,tweet,label,clean_tweet
id,,,
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,-1,vinco tresorpack difficulti object disassembl ...
1,glad i dot have taks tomorrow ! ! #thankful #s...,-1,glad dot tak tomorrow #thank #startho
2,1-3 vs celtics in the regular season = were fu...,-1,vs celtic regular season fuck play playoff
3,<user> i could actually kill that girl i'm so ...,-1,could actual kill girl sorri
4,<user> <user> <user> i find that very hard to ...,-1,find hard believ im afraid


In [23]:
import csv, os

def submission_to_csv(predictions, filename):
    if not os.path.exists(PREDS_FOLDER):
        os.makedirs(PREDS_FOLDER)
    predictions.to_csv(PREDS_FOLDER + filename + ".csv")

### Bag of Words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['clean_tweet'])

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_bow = bow

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'], random_state=42, test_size=0.3)


In [5]:
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

0.7786465875084468

### TF IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(train['clean_tweet'])

In [7]:
train_tfidf = tfidf

xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

lreg.fit(xtrain_tfidf, ytrain)

prediction = lreg.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)

0.7774145944529492

### Fasttext (embedding)

In [4]:
from embeddings import *

#model = fasttext_model(train, model = "skipgram")
#print(model.words)
# TODO use embedding

### Fasttext (prediction)

#### on "clean_tweet"

In [ ]:
best_precision = 0;
i_best = -1;

for i in range(0,100):
    train_name, test_name = create_labeled_csv(train, tweets_col = "clean_tweet", split_test = True)
    classifier = fasttext.supervised(train_name, 'model_supervised', label_prefix='__label__')

    result = classifier.test(test_name)
    #print('P@1:', result.precision)
    #print('R@1:', result.recall)
    #print('Number of examples:', result.nexamples)
    if result.precision > best_precision:
        print('\tprecision: ' + str(best_precision) + ' --> ' + str(result.precision))
        best_precision = result.precision
        i_best = i
        
        labels = classifier.predict(load_txt(TEST_DATA))
        #flatten the result labels
        labels = [y for x in labels for y in x]
        pred_df= pd.DataFrame({'Prediction': labels})
        pred_df.index.name = 'Id' # rename id column
        pred_df.index += 1 #shift to correspond to sample submission
        submission_to_csv(pred_df, "fasttext_" + str(best_precision))
        
        
        
print('Best precision: ', best_precision)

	precision: 0 --> 0.48153796155096124
	precision: 0.48153796155096124 --> 0.5554111147221319
	precision: 0.5554111147221319 --> 0.6379590510237244
	precision: 0.6379590510237244 --> 0.6888327791805204
	precision: 0.6888327791805204 --> 0.7803804904877378
	precision: 0.7803804904877378 --> 0.8666283342916427


In [21]:
labels = classifier.predict(load_txt(TEST_DATA))
#flatten the result labels
labels = [y for x in labels for y in x]
pred_df= pd.DataFrame({'Prediction': labels})
pred_df.index.name = 'Id' # rename id column
pred_df.index += 1 #shift to correspond to sample submission
pred_df.head()

,prediction
1,-1
2,1
3,-1
4,1
5,1


In [13]:
print(labels)

['-1', '1', '-1', '1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '1', '1', '-1', '1', '-1', '1', '-1', '1', '-1', '1', '1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '-1', '-1', '-1', '-1', '1', '1', '1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '1', '1', '-1', '-1', '1', '-1', '1', '1', '1', '-1', '1', '1', '1', '-1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '1', '1', '1', '-1', '1', '1', '1', '1', '-1', '1', '1', '1', '1', '-1', '-1', '-1', '1', '-1', '-1', '1', '1', '-1', '1', '-1', '1', '1', '-1', '1', '-1', '1', '1', '1', '1', '1', '1', '-1', '1', '-1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '-1', '-1', '1', '1', '1', '-1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '-1', '1', '-1', '1',

#### on "tweet"

In [30]:
best_precision = 0;
i_best = -1;

for i in range(0,100):
    train_name, test_name = create_labeled_csv(train, tweets_col = "tweet", split_test = True)
    classifier = fasttext.supervised(train_name, 'model_supervised', label_prefix='__label__')

    result = classifier.test(test_name)
    #print('P@1:', result.precision)
    #print('R@1:', result.recall)
    #print('Number of examples:', result.nexamples)
    if result.precision > best_precision:
        print('\tprecision: ' + str(best_precision) + ' --> ' + str(result.precision))
        best_precision = result.precision
        i_best = i
print('Best precision: ', best_precision)

	precision: 0 --> 0.6857328566785831
	precision: 0.6857328566785831 --> 0.7752556186095347
	precision: 0.7752556186095347 --> 0.8722281942951426
	precision: 0.8722281942951426 --> 0.8911027224319392
	precision: 0.8911027224319392 --> 0.9315767105822355
Best precision:  0.9315767105822355


In [11]:
# TODO fix MemoryError 
#classifier2 = fasttext.load_model('models/word_deps_sg_500d/words500.npy')

In [12]:
#result = classifier.test('./data/fasttext_train.txt')
#print('P@1:', result.precision)
#print('R@1:', result.recall)
#print('Number of examples:', result.nexamples)